In [131]:
# !pip install amplify
# !pip install amplify[extra]

In [132]:
import numpy as np
from amplify import BinaryPoly, BinaryMatrix, BinaryQuadraticModel, gen_symbols, Solver
from amplify.client import FixstarsClient
from amplify.client.ocean import DWaveSamplerClient, LeapHybridSamplerClient


In [133]:
n = 10 # number of inputs

N = np.random.randint(1,100,n)
print('Input numbers are:\n',N)

Input numbers are:
 [93 49 35 57 30 73  2 70 24 16]


In [134]:
def QUBO(N):
    k = N.shape[0]
    m = BinaryMatrix(k)
    for i in range(k):
        for j in range(i,k):
            m[i,j] = (-1)**(N[i] + N[j])
    return (-1)*m


qubo = QUBO(N)
print(qubo.to_numpy())

[[-1. -1. -1. -1.  1. -1.  1.  1.  1.  1.]
 [ 0. -1. -1. -1.  1. -1.  1.  1.  1.  1.]
 [ 0.  0. -1. -1.  1. -1.  1.  1.  1.  1.]
 [ 0.  0.  0. -1.  1. -1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0. -1.  1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0. -1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]]


In [135]:
model = BinaryQuadraticModel(qubo)

client = FixstarsClient()
client.token = "mBle_KURyYutT.NDbsTpovEeCWf-vScH"
client.parameters.timeout = 1000
# client.parameters.outputs.duplicate = True  
client.parameters.outputs.num_outputs = 0  

######### client hardware property ########
print('Total number of allowed bits:',client.num_bits)
print('client API:',client.url)
print('Fixstars client version:',client.version,'\n')


################## Solution #########################

solver = Solver(client)
result = solver.solve(model)
for solution in result:
    print(f"values = {list(solution.values.values())} energy = {solution.energy} frequency = {solution.frequency}")
print('numbers=',N)

Total number of allowed bits: 65536
client API: https://optigan.fixstars.com/
Fixstars client version: v0.3.2 

values = [0, 0, 0, 0, 1, 0, 1, 1, 1, 1] energy = -15.0 frequency = 1
numbers= [93 49 35 57 30 73  2 70 24 16]


In [136]:
############# Solving with Dwave client  ##################



client = DWaveSamplerClient()
client.token = "DEV-d1f6e1d1c38fbe9219ce7713e110736473b657a0"
client.solver = "Advantage_system1.1"
client.parameters.num_reads = 1000 


############   Solution   ######################
solver = Solver(client)
result = solver.solve(model)
for solution in result:
    print(f"nvalues = {list(solution.values.values())} energy = {solution.energy} frequency = {solution.frequency}")

print('numbers=',N)

nvalues = [0, 0, 0, 0, 1, 0, 1, 1, 1, 1] energy = -15.0 frequency = 149
nvalues = [1, 1, 1, 1, 0, 1, 0, 0, 0, 0] energy = -15.0 frequency = 851
numbers= [93 49 35 57 30 73  2 70 24 16]


In [137]:
############# Solving with Dwave hybrid client  ##################



client = LeapHybridSamplerClient()
client.token = "DEV-d1f6e1d1c38fbe9219ce7713e110736473b657a0"
client.solver = "hybrid_binary_quadratic_model_version2"
# client.parameters.time_limit = 3

############   Solution   ######################
solver = Solver(client)
result = solver.solve(model)
for solution in result:
    print(f"nvalues = {list(solution.values.values())} energy = {solution.energy} frequency = {solution.frequency}")

print('numbers=',N)

nvalues = [1, 1, 1, 1, 0, 1, 0, 0, 0, 0] energy = -15.0 frequency = 1
numbers= [93 49 35 57 30 73  2 70 24 16]
